<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/dialectic_bundle_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Provenance-first dialectic agents: reproducible run + portable bundle + optional encryption.
No external dependencies. Python 3.9+.

Usage examples:
  Run (default, with 6 turns) and produce artifacts in ./runs/run-<timestamp>-<label>:
    python dialectic_bundle.py run --seed-context "Limits of computation in multiverse physics" --rounds 6 --label multiverse-limits --seed 4242

  Run and encrypt a portable bundle with a passphrase:
    python dialectic_bundle.py run --seed-context "Limits..." --passphrase "your passphrase"

  Verify a manifest against files on disk:
    python dialectic_bundle.py verify --manifest /path/to/manifest.yaml

  Decrypt an encrypted bundle to a directory:
    python dialectic_bundle.py decrypt --in-bundle /path/to/bundle.sfp --out-dir ./recovered --passphrase "your passphrase"
"""
from __future__ import annotations

import argparse
import base64
import binascii
import csv
import datetime as dt
import getpass
import hashlib
import io
import json
import os
import random
import re
import shutil
import sys
import tarfile
import textwrap
import time
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import Callable, Iterable, List, Optional, Sequence, Tuple

# ------------------------------------------------------------
# RNG, hashing, integrity utilities
# ------------------------------------------------------------

def now_iso() -> str:
    return dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc).isoformat().replace("+00:00", "Z")

def hash_bytes(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()

def hash_file(path: Path) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1 << 16), b""):
            h.update(chunk)
    return h.hexdigest()

def ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)

def try_import_numpy_seed(seed: int) -> Optional[str]:
    try:
        import numpy as np  # type: ignore
        np.random.seed(seed)
        return "numpy"
    except Exception:
        return None

def deterministic_seed(seed: Optional[int]) -> int:
    if seed is None:
        # Derive from current time but recorded in config for reproducibility
        seed = int(time.time_ns() % (2**31 - 1))
    random.seed(seed)
    try_import_numpy_seed(seed)
    return seed

# ------------------------------------------------------------
# Lightweight passphrase-based stream cipher (stdlib only)
# NOTE: This is for artifact protection/convenience, not a
# replacement for audited cryptography. Uses scrypt KDF, a
# BLAKE2b-based PRF keystream, XOR, and HMAC-SHA256.
# ------------------------------------------------------------

SFP_MAGIC = b"SFP1"  # SpiralFrame Pack v1 (conceptual tag)

def kdf_scrypt(passphrase: str, salt: bytes, dklen: int = 64) -> bytes:
    return hashlib.scrypt(
        passphrase.encode("utf-8"),
        salt=salt,
        n=2**14,
        r=8,
        p=1,
        maxmem=0,
        dklen=dklen,
    )

def prf_block(key: bytes, nonce: bytes, counter: int, outlen: int = 64) -> bytes:
    # BLAKE2b keyed PRF over (nonce || counter_le_8)
    c_bytes = counter.to_bytes(8, "little")
    h = hashlib.blake2b(key=key, digest_size=outlen)
    h.update(nonce)
    h.update(c_bytes)
    return h.digest()

def xor_stream(data: bytes, key: bytes, nonce: bytes) -> bytes:
    out = bytearray(len(data))
    block_size = 64
    blocks = (len(data) + block_size - 1) // block_size
    for i in range(blocks):
        ks = prf_block(key, nonce, i, outlen=block_size)
        start = i * block_size
        end = min(start + block_size, len(data))
        for j in range(start, end):
            out[j] = data[j] ^ ks[j - start]
    return bytes(out)

def hmac_sha256(key: bytes, data: bytes) -> bytes:
    # Simple HMAC implementation using hashlib (key pad)
    block_size = 64
    if len(key) > block_size:
        key = hashlib.sha256(key).digest()
    key = key.ljust(block_size, b"\x00")
    o_key_pad = bytes((b ^ 0x5C) for b in key)
    i_key_pad = bytes((b ^ 0x36) for b in key)
    inner = hashlib.sha256(i_key_pad + data).digest()
    return hashlib.sha256(o_key_pad + inner).digest()

def encrypt_bytes(data: bytes, passphrase: str) -> bytes:
    salt = os.urandom(16)
    nonce = os.urandom(16)[:12]
    dk = kdf_scrypt(passphrase, salt, dklen=64)
    enc_key = dk[:32]
    mac_key = dk[32:]
    ct = xor_stream(data, enc_key, nonce)
    tag = hmac_sha256(mac_key, SFP_MAGIC + salt + nonce + ct)
    return SFP_MAGIC + salt + nonce + ct + tag

def decrypt_bytes(packed: bytes, passphrase: str) -> bytes:
    if len(packed) < 4 + 16 + 12 + 32:
        raise ValueError("Bundle too short / invalid format")
    magic = packed[:4]
    if magic != SFP_MAGIC:
        raise ValueError("Bad magic header")
    salt = packed[4:20]
    nonce = packed[20:32]
    tag = packed[-32:]
    ct = packed[32:-32]
    dk = kdf_scrypt(passphrase, salt, dklen=64)
    enc_key = dk[:32]
    mac_key = dk[32:]
    expected = hmac_sha256(mac_key, SFP_MAGIC + salt + nonce + ct)
    if not hmac_compare(expected, tag):
        raise ValueError("Authentication failed (bad passphrase or corrupted data)")
    pt = xor_stream(ct, enc_key, nonce)
    return pt

def hmac_compare(a: bytes, b: bytes) -> bool:
    # Constant-time compare
    if len(a) != len(b):
        return False
    result = 0
    for x, y in zip(a, b):
        result |= x ^ y
    return result == 0

# ------------------------------------------------------------
# Question generator and scorer (heuristics; replaceable)
# ------------------------------------------------------------

def generate_open_ended_questions(prompt: str, n: int = 1) -> List[str]:
    """
    Heuristic deepener: converts an input question into more challenging forms.
    Deterministic given RNG seed and order. Replace with your model if desired.
    """
    base = extract_quoted_question(prompt) or prompt.strip()
    outs = []
    for i in range(n):
        spice = random.choice([
            "quantify", "operationalize", "falsify", "counterexample",
            "trade-offs", "measure problem", "resource bounds",
            "no-free-lunch", "observer selection", "thermodynamic costs",
            "computability", "complexity classes", "oracle access", "noise",
        ])
        frame = random.choice([
            "Under what concrete assumptions would this break down, and how would you know?",
            "Make it testable: specify observables, thresholds, and a falsification path.",
            "Map the edge cases: what’s the tightest counterexample you can construct?",
            "Turn it numerical: propose metrics and target ranges worth optimizing.",
            "Expose hidden priors: which ones change the conclusion the most?",
            "Interrogate scaling laws: how do asymptotics mislead at real-world scales?",
            "Force a trade: what do you sacrifice to achieve the claimed gain?",
            "Name the invariants: what stays constant across universes and why?",
        ])
        template = f"{tighten(base)} — {spice.capitalize()} it. {frame}"
        outs.append(template)
    return outs

def extract_quoted_question(text: str) -> Optional[str]:
    m = re.search(r"'([^']+)'", text) or re.search(r"\"([^\"]+)\"", text)
    return m.group(1) if m else None

def tighten(q: str) -> str:
    q = re.sub(r"\s+", " ", q).strip(" ?")
    # Light rewriter to add precision prompts
    prefixes = [
        "Reframe precisely:",
        "Sharpen the claim:",
        "State the decisive version:",
        "Pose the smallest hard question:",
    ]
    return f"{random.choice(prefixes)} {q}?"

KEYWEIGHTS = {
    "quantify": 1.0, "operationalize": 1.2, "falsify": 1.2, "counterexample": 1.1,
    "trade-off": 0.9, "trade offs": 0.9, "trade-offs": 0.9, "measure": 0.8,
    "resource": 0.8, "complexity": 1.0, "computability": 1.0, "oracle": 0.8,
    "noise": 0.7, "scaling": 0.7, "assumptions": 0.7, "invariants": 0.6,
    "threshold": 0.6, "observable": 0.6, "falsification": 1.2,
}

def score_question(q: str) -> float:
    """
    Heuristic 'complexity' score in [0,1]: length + key terms + structure.
    Deterministic. Replace with your evaluator if desired.
    """
    ql = q.lower()
    length_term = min(len(q) / 200.0, 1.0) * 0.35
    keys = sum(w for k, w in KEYWEIGHTS.items() if k in ql)
    key_term = min(keys / 6.0, 1.0) * 0.45
    structure = 0.2 if any(x in ql for x in ["under what", "how would", "make it", "map the", "turn it", "expose", "interrogate", "force a", "name the"]) else 0.0
    return max(0.0, min(1.0, length_term + key_term + structure))

# ------------------------------------------------------------
# Agents and dialectic
# ------------------------------------------------------------

@dataclass
class SelfReflectiveAgent:
    generator_fn: Callable[[str, int], List[str]]
    scorer_fn: Callable[[str], float]
    threshold: float = 0.8

    def generate_initial_question(self, seed_context: str) -> str:
        # Seed into a crisp initial probe
        base = f"What is the most decision-relevant question about {seed_context}?"
        return self.generator_fn(base, n=1)[0]

class DialecticAgent(SelfReflectiveAgent):
    def challenge_question(self, other_question: str) -> str:
        prompt = (
            "As an advanced thinker, respond to the following question with a deeper, more challenging version: "
            f"'{other_question}'"
        )
        return self.generator_fn(prompt, n=1)[0]

def dialectic_interaction(agent1: DialecticAgent, agent2: DialecticAgent, seed_context: str, rounds: int = 5):
    q = agent1.generate_initial_question(seed_context)
    history = [("Agent1", q)]
    for i in range(rounds):
        challenger = agent2 if i % 2 == 0 else agent1
        label = "Agent2" if i % 2 == 0 else "Agent1"
        q = challenger.challenge_question(q)
        history.append((label, q))
    return history

# ------------------------------------------------------------
# Rendering and artifacts
# ------------------------------------------------------------

def write_csv(path: Path, rows: List[Tuple[int, str, str, float]], seed_context: str) -> None:
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["seed_context", seed_context])
        w.writerow(["turn_index", "speaker", "question", "score"])
        for t, speaker, question, score in rows:
            w.writerow([t, speaker, question, f"{score:.4f}"])

def render_svg_line_chart(scores: List[float], width: int = 880, height: int = 300, margin: int = 40) -> str:
    if not scores:
        scores = [0.0]
    n = len(scores)
    w, h, m = width, height, margin
    inner_w = w - 2*m
    inner_h = h - 2*m
    # Map points
    pts = []
    for i, s in enumerate(scores):
        x = m + (inner_w * (i / max(1, n-1)))
        y = m + (inner_h * (1.0 - s))
        pts.append((x, y))
    # Polyline
    poly = " ".join(f"{x:.1f},{y:.1f}" for x, y in pts)
    # Points
    circles = "\n".join(f'<circle cx="{x:.1f}" cy="{y:.1f}" r="3" fill="#1a73e8"/>' for x, y in pts)
    # Axes and grid
    grid = []
    for k in range(0, 11):
        y = m + inner_h * (k / 10.0)
        grid.append(f'<line x1="{m}" y1="{y:.1f}" x2="{w-m}" y2="{y:.1f}" stroke="#eee"/>')
        label = f"{1.0 - k/10.0:.1f}"
        grid.append(f'<text x="{m-10}" y="{y+4:.1f}" font-size="10" text-anchor="end" fill="#777">{label}</text>')
    # X labels
    xlabels = []
    for i, (x, _) in enumerate(pts):
        xlabels.append(f'<text x="{x:.1f}" y="{h-m+14}" font-size="10" text-anchor="middle" fill="#777">{i}</text>')
    svg = f"""<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}" viewBox="0 0 {w} {h}">
  <rect x="0" y="0" width="{w}" height="{h}" fill="#fff"/>
  <g>
    <line x1="{m}" y1="{m}" x2="{m}" y2="{h-m}" stroke="#333" />
    <line x1="{m}" y1="{h-m}" x2="{w-m}" y2="{h-m}" stroke="#333" />
    {''.join(grid)}
    <polyline fill="none" stroke="#1a73e8" stroke-width="2" points="{poly}"/>
    {circles}
    {''.join(xlabels)}
    <text x="{w/2:.1f}" y="22" font-size="14" text-anchor="middle" fill="#111">Question complexity over turns</text>
    <text x="{m}" y="{m-10}" font-size="10" fill="#555">Score (0–1)</text>
    <text x="{w-m}" y="{h-m+28}" font-size="10" text-anchor="end" fill="#555">Turn index</text>
  </g>
</svg>"""
    return svg

def write_text(path: Path, text: str) -> None:
    path.write_text(text, encoding="utf-8")

def write_json(path: Path, obj) -> None:
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")

def derive_parable(history: List[Tuple[int, str, str, float]], seed_context: str) -> dict:
    motifs = []
    if any("falsif" in q.lower() for _, _, q, _ in history):
        motifs.append("falsifiability")
    if any("measure" in q.lower() for _, _, q, _ in history):
        motifs.append("measure problem")
    if any("resource" in q.lower() or "thermo" in q.lower() for _, _, q, _ in history):
        motifs.append("resource bounds")
    if any("complex" in q.lower() or "compute" in q.lower() for _, _, q, _ in history):
        motifs.append("computational limits")
    text = (
        f"In the hall of questions about {seed_context}, two voices traded sharper blades. "
        f"Each turn, the claims grew measurable; each edge exposed a hidden cost. "
        f"The parable’s lesson: name your priors, price your miracles, and leave a thread for refutation."
    )
    return {
        "seed_context": seed_context,
        "motifs": motifs or ["rigor", "measurability", "humility"],
        "moral": "Truth likes receipts. Ask so that answers can break.",
        "parable": text,
    }

def build_index_html(seed_context: str, svg_str: str, rows: List[Tuple[int, str, str, float]]) -> str:
    table_rows = "\n".join(
        f"<tr><td>{t}</td><td>{speaker}</td><td>{html_escape(q)}</td><td>{score:.4f}</td></tr>"
        for t, speaker, q, score in rows
    )
    html = f"""<!doctype html>
<html lang="en">
<meta charset="utf-8">
<title>Dialectic transcript — {html_escape(seed_context)}</title>
<meta name="viewport" content="width=device-width, initial-scale=1" />
<style>
  body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, 'Noto Sans', Arial, sans-serif; margin: 20px; color: #111; }}
  h1, h2 {{ margin: 0.4em 0; }}
  .grid {{ display: grid; grid-template-columns: 1fr; gap: 24px; }}
  @media (min-width: 900px) {{ .grid {{ grid-template-columns: 1fr 1fr; }} }}
  table {{ border-collapse: collapse; width: 100%; font-size: 14px; }}
  th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
  th {{ background: #fafafa; text-align: left; }}
  .muted {{ color: #666; font-size: 12px; }}
  .card {{ border: 1px solid #eee; padding: 12px; border-radius: 8px; background: #fff; }}
  code {{ background: #f6f8fa; padding: 2px 4px; border-radius: 4px; }}
</style>
<body>
  <h1>Dialectic transcript</h1>
  <p class="muted">Seed context: <code>{html_escape(seed_context)}</code></p>
  <div class="grid">
    <div class="card">
      <h2>Complexity chart</h2>
      {svg_str}
    </div>
    <div class="card">
      <h2>Turns</h2>
      <table>
        <thead><tr><th>Turn</th><th>Speaker</th><th>Question</th><th>Score</th></tr></thead>
        <tbody>
          {table_rows}
        </tbody>
      </table>
    </div>
  </div>
  <p class="muted">Portable artifact generated deterministically given the recorded seed. Integrity receipts live in manifest.yaml.</p>
</body>
</html>"""
    return html

def html_escape(s: str) -> str:
    return (s.replace("&", "&amp;")
             .replace("<", "&lt;")
             .replace(">", "&gt;")
             .replace('"', "&quot;")
             .replace("'", "&#39;"))

# ------------------------------------------------------------
# Manifest and plan
# ------------------------------------------------------------

def build_manifest(run_dir: Path, files: List[Path], meta: dict) -> str:
    lines = []
    lines.append("version: 1")
    for k, v in meta.items():
        lines.append(f"{k}: {yaml_scalar(v)}")
    lines.append("files:")
    for p in files:
        rel = p.relative_to(run_dir).as_posix()
        size = p.stat().st_size
        sha = hash_file(p)
        lines.append(f"  - path: {yaml_scalar(rel)}")
        lines.append(f"    bytes: {size}")
        lines.append(f"    sha256: {sha}")
        lines.append(f"    created_at: {yaml_scalar(now_iso())}")
        lines.append(f"    content_type: {yaml_scalar(guess_content_type(p))}")
    # Determinism receipt
    all_hash = hash_bytes("\n".join(sorted(hash_file(p) for p in files)).encode("utf-8"))
    lines.append(f"determinism_hash: {all_hash}")
    return "\n".join(lines) + "\n"

def yaml_scalar(v) -> str:
    if isinstance(v, (int, float)):
        return str(v)
    s = str(v)
    if re.search(r"[:#\n\"']", s) or s.strip() != s or s == "" or s.startswith(("true","false","null")):
        s = s.replace("\\", "\\\\").replace('"', '\\"')
        return f'"{s}"'
    return s

def guess_content_type(p: Path) -> str:
    if p.name.endswith(".json"): return "application/json"
    if p.name.endswith(".csv"): return "text/csv"
    if p.name.endswith(".svg"): return "image/svg+xml"
    if p.name.endswith(".html"): return "text/html"
    if p.name.endswith(".yaml") or p.name.endswith(".yml"): return "application/yaml"
    if p.name.endswith(".tar"): return "application/x-tar"
    if p.name.endswith(".gz"): return "application/gzip"
    if p.name.endswith(".sfp"): return "application/octet-stream"
    return "text/plain"

# ------------------------------------------------------------
# CLI commands: run, verify, decrypt
# ------------------------------------------------------------

def cmd_run(args: argparse.Namespace) -> int:
    # ...all your earlier setup logic...
    seed = deterministic_seed(args.seed)
    numpy_state = try_import_numpy_seed(seed)
    label = args.label or "dialectic"
    run_id = f"run-{timestamp}-{slugify(label)}"
    run_dir = Path(args.outdir or "./runs") / run_id
    ensure_dir(run_dir)

    # Agents
    agent_a = DialecticAgent(generator_fn=generate_open_ended_questions, scorer_fn=score_question, threshold=0.8)
    agent_b = DialecticAgent(generator_fn=generate_open_ended_questions, scorer_fn=score_question, threshold=0.8)

    # Dialectic
    history_pairs = dialectic_interaction(agent_a, agent_b, args.seed_context, rounds=args.rounds)
    # Assemble with scores
    rows = []
    for idx, (speaker, q) in enumerate(history_pairs):
        score = score_question(q)
        rows.append((idx, speaker, q, score))

    # Artifacts
    # 1) out.csv
    out_csv = run_dir / "out.csv"
    write_csv(out_csv, rows, args.seed_context)

    # 2) out.svg
    svg = render_svg_line_chart([r[3] for r in rows])
    out_svg = run_dir / "out.svg"
    write_text(out_svg, svg)

    # 3) plan.json
    plan = {
        "run_id": run_id,
        "created_at": now_iso(),
        "steps": [
            {"name": "seed_rng", "seed": seed, "numpy_seeded": bool(numpy_state)},
            {"name": "generate_questions", "rounds": args.rounds},
            {"name": "score_questions"},
            {"name": "write_csv_svg_html_json_yaml"},
            {"name": "bundle_and_encrypt", "enabled": bool(args.passphrase)},
        ],
    }
    write_json(run_dir / "plan.json", plan)

    # 4) parable.json
    parable = derive_parable(rows, args.seed_context)
    write_json(run_dir / "parable.json", parable)

    # 5) config.json
cfg = {
    "seed_context": args.seed_context,
    "rounds": args.rounds,
    "seed": seed,
    "label": label,
    "run_id": run_id,
    "created_at": now_iso(),
    "numpy_seeded": bool(numpy_state),
    "env": {
        "python": sys.version.split()[0],
        "platform": sys.platform,
    },
}
write_json(run_dir / "config.json", cfg)

# 6) index.html
html = build_index_html(args.seed_context, svg, rows)
write_text(run_dir / "index.html", html)

# 7) manifest.yaml
files = [
    run_dir / "plan.json",
    run_dir / "config.json",
    run_dir / "out.csv",
    run_dir / "out.svg",
    run_dir / "parable.json",
    run_dir / "index.html",
]
manifest_meta = {
    "run_id": run_id,
    "created_at": now_iso(),
    "label": label,
    "seed": seed,
    "seed_context": args.seed_context,
    "notes": "Deterministic artifacts given the same seed and code. Encryption, if used, applies to the packed bundle only.",
}
manifest_text = build_manifest(run_dir, files, manifest_meta)
write_text(run_dir / "manifest.yaml", manifest_text)

# 8) Optional pack (.tar) and encrypt (.sfp)
bundle_tar = run_dir / f"{run_id}.tar"
with tarfile.open(bundle_tar, "w") as tf:
    for p in files + [run_dir / "manifest.yaml"]:
        tf.add(p, arcname=p.name)

# Integrity receipt for the tar itself
tar_sha = hash_file(bundle_tar)
write_text(run_dir / "integrity.txt", f"bundle_tar_sha256: {tar_sha}\n")

if args.passphrase:
    enc = encrypt_bytes(bundle_tar.read_bytes(), args.passphrase)
    bundle_enc = run_dir / f"{run_id}.sfp"
    bundle_enc.write_bytes(enc)
    # Small sidecar info
    write_text(
        run_dir / "encryption.txt",
        "cipher: scrypt + BLAKE2b-PRF XOR stream + HMAC-SHA256\n"
        "note: convenience layer; not a replacement for audited crypto\n"
    )

# Dry-run note (files still written; dry-run here means 'do not delete temp', kept simple)
return 0

def cmd_verify(args: argparse.Namespace) -> int:
    manifest_path = Path(args.manifest)
    run_dir = manifest_path.parent
    manifest_lines = manifest_path.read_text(encoding="utf-8").splitlines()

    # Parse simple YAML for 'files' section
    files = []
    in_files = False
    current = None

    for line in manifest_lines:
        if line.strip() == "files:":
            in_files = True
            continue
        if not in_files:
            continue

        if line.startswith("  - "):
            # Commit previous record
            if current:
                files.append(current)
            current = {}
            # Handle inline "  - key: value" (especially "path")
            rest = line[4:].strip()
            if ": " in rest:
                k, v = rest.split(": ", 1)
                current[k.strip()] = v.strip().strip('"')
            continue

        if line.startswith("    ") and current is not None:
            # Indented key/value lines inside current entry
            kv = line.strip()
            if ": " in kv:
                k, v = kv.split(": ", 1)
                current[k.strip()] = v.strip().strip('"')
            continue

        # End of files block
        if current:
            files.append(current)
            current = None
        break

    if current:
        files.append(current)

    ok = True
    for f in files:
        rel = f.get("path")
        expected = f.get("sha256")
        if not rel or not expected:
            print(f"[SKIP] malformed entry: {f}")
            ok = False
            continue
        p = run_dir / rel
        if not p.exists():
            print(f"[MISSING] {p}")
            ok = False
            continue
        actual = hash_file(p)
        if actual != expected:
            print(f"[MISMATCH] {p} sha256 {actual} != {expected}")
            ok = False
        else:
            print(f"[OK] {p} sha256 matches")

    print("VERIFY:", "PASS" if ok else "FAIL")
    return 0 if ok else 2

def cmd_decrypt(args: argparse.Namespace) -> int:
    in_path = Path(args.in_bundle)
    if not in_path.exists():
        print(f"Not found: {in_path}", file=sys.stderr)
        return 2

    passphrase = args.passphrase or getpass.getpass("Passphrase: ")
    packed = in_path.read_bytes()
    try:
        plain = decrypt_bytes(packed, passphrase)
    except Exception as e:
        print(f"Decrypt error: {e}", file=sys.stderr)
        return 3

    out_dir = Path(args.out_dir or "./recovered")
    ensure_dir(out_dir)

    # Extract tar
    bio = io.BytesIO(plain)
    with tarfile.open(fileobj=bio, mode="r:*") as tf:
        tf.extractall(out_dir)

    print(f"Decrypted and extracted to: {out_dir}")
    return 0

def slugify(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"[^a-z0-9\-]+", "-", s)
    s = re.sub(r"-{2,}", "-", s).strip("-")
    return s or "run"

# ---------------------------
# Main
# ---------------------------

def build_arg_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="Provenance-first dialectic runner and bundler")
    sub = p.add_subparsers(dest="cmd", required=True)

    pr = sub.add_parser("run", help="Run the dialectic and emit artifacts")
    pr.add_argument("--seed-context", dest="seed_context", required=True, help="Seed topic/context to start the dialectic")
    pr.add_argument("--rounds", type=int, default=6, help="Number of challenge rounds (after the initial question)")
    pr.add_argument("--seed", type=int, default=None, help="RNG seed for determinism")
    pr.add_argument("--label", type=str, default=None, help="Label for run id")
    pr.add_argument("--outdir", type=str, default=None, help="Base output directory (default: ./runs)")
    pr.add_argument("--passphrase", type=str, default=None, help="Optional passphrase to encrypt the bundle (.sfp)")

    pv = sub.add_parser("verify", help="Verify a manifest against current files")
    pv.add_argument("--manifest", required=True, help="Path to manifest.yaml to verify")

    pd = sub.add_parser("decrypt", help="Decrypt a .sfp bundle and extract")
    pd.add_argument("--in-bundle", dest="in_bundle", required=True, help="Path to encrypted .sfp bundle")
    pd.add_argument("--out-dir", dest="out_dir", required=False, help="Directory to extract into")
    pd.add_argument("--passphrase", required=False, help="Passphrase (prompted if not provided)")

    return p

if __name__ == "__main__":
    ap = build_arg_parser()
    args = ap.parse_args()
    if args.cmd == "run":
        sys.exit(cmd_run(args))
    elif args.cmd == "verify":
        sys.exit(cmd_verify(args))
    elif args.cmd == "decrypt":
        sys.exit(cmd_decrypt(args))
    else:
        ap.print_help()
        sys.exit(1)